In [6]:
from analyze import *

In [8]:
evaluator = TestEvaluator('../../data/raw/openja/lightfm_demo')
evaluator.load_checklist('../../checklist/checklist.yaml')

evaluator.evaluate()

  0%|                                                                                                                                                                                                                   | 0/3 [00:00<?, ?it/s]

../../data/raw/openja/lightfm_demo/tests/test_cross_validation.py
# splits: 3


 33%|███████████████████████████████████████████████████████████████████▋                                                                                                                                       | 1/3 [00:29<00:58, 29.32s/it]

../../data/raw/openja/lightfm_demo/tests/test_evaluation.py
# splits: 3


 67%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                   | 2/3 [00:59<00:30, 30.07s/it]

../../data/raw/openja/lightfm_demo/tests/test_data.py
# splits: 3


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:29<00:00, 29.97s/it]


In [ ]:
evaluator.get_completeness_score()